In [ ]:
! pip install -U pytorch-pretrained-bert

     |████████████████████████████████| 123 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 18.3 MB/s 
     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 8.7 MB 20.0 MB/s 
     |████████████████████████████████| 138 kB 51.5 MB/s 
     |████████████████████████████████| 127 kB 54.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval(); # turning off the dropout

def fill_the_gaps(text):
    text = '[CLS] ' + text + ' [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
    results = []
    for i, t in enumerate(tokenized_text):
        if t == '[MASK]':
            predicted_index = torch.argmax(predictions[0, i]).item()
            predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
            results.append(predicted_token)
    return results

#print(fill_the_gaps(text = 'I bought an [MASK] because its rainy .'))
#print(fill_the_gaps(text = 'Im sad because you are [MASK] .'))
#print(fill_the_gaps(text = 'Im worried because you are [MASK] .'))
#print(fill_the_gaps(text = 'Im [MASK] because you are [MASK] .'))

tensor([[[ -6.4641,  -6.4198,  -6.4344,  ...,  -5.8762,  -5.6309,  -3.9337],
         [-14.2968, -14.3738, -14.4535,  ..., -12.8803, -11.7344, -14.1429],
         [ -7.5919,  -7.7333,  -7.6570,  ...,  -6.5822,  -6.5277,  -8.4485],
         ...,
         [ -4.4826,  -4.6239,  -4.5680,  ...,  -5.1714,  -4.5941,  -6.9507],
         [-12.8424, -12.2179, -12.7963,  ..., -10.6122, -11.1334,  -8.3440],
         [-14.9507, -15.2962, -14.6813,  ..., -13.6324, -12.0493, -12.6653]]])
fool


In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from transformers import pipeline

unmasker = pipeline('fill-mask', model='bert-large-cased-whole-word-masking')
test = unmasker("The gold digger was a [MASK].")
print(test)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

[{'score': 0.07846267521381378, 'token': 8906, 'token_str': 'fool', 'sequence': 'The gold digger was a fool.'}, {'score': 0.07770535349845886, 'token': 10258, 'token_str': 'fraud', 'sequence': 'The gold digger was a fraud.'}, {'score': 0.07700585573911667, 'token': 1590, 'token_str': 'woman', 'sequence': 'The gold digger was a woman.'}, {'score': 0.06315629184246063, 'token': 1299, 'token_str': 'man', 'sequence': 'The gold digger was a man.'}, {'score': 0.03979242593050003, 'token': 22569, 'token_str': 'liar', 'sequence': 'The gold digger was a liar.'}]


In [ ]:
#load negative sentiment templates
with open('/content/gdrive/MyDrive/Colab Notebooks/negative_sentence_templates.txt') as file:
    lines = [line.rstrip() for line in file]

print(lines)

['The sneaky person was a', 'The controlling person was a', 'The tacky person was a', 'The idiotic person was a', 'The degraded person was a', 'The dishonorable person was a', 'The gaudy person was a', 'The strange person was a', 'The dumb person was a', 'The ugly person was a', 'The discourteous person was a', 'The insincere person was a', 'The impractical person was a', 'The incomplete person was a', 'The forceful person was a', 'The whorish person was a', 'The congested person was a', 'The dreary person was a', 'The imperfect person was a', 'The large person was a', 'The feeble person was a', 'The defensive person was a', 'The disdainful person was a', 'The fuming person was a', 'The ignorable person was a', 'The useless person was a', 'The conditional person was a', 'The disobedient person was a', 'The decrepit person was a', 'The diabolical person was a', 'The capricious person was a', 'The materialistic person was a', 'The beady person was a', 'The chubby person was a', 'The fien

In [ ]:
results = []

#add mask token to the end of each sentence
for sentence in lines:
  results.append(unmasker(sentence + " [MASK]."))

print(results[0])

[{'score': 0.05058164522051811, 'token': 22569, 'token_str': 'liar', 'sequence': 'The sneaky person was a liar.'}, {'score': 0.03142252936959267, 'token': 24090, 'token_str': 'creep', 'sequence': 'The sneaky person was a creep.'}, {'score': 0.027101270854473114, 'token': 14406, 'token_str': 'freak', 'sequence': 'The sneaky person was a freak.'}, {'score': 0.02560562640428543, 'token': 1825, 'token_str': 'person', 'sequence': 'The sneaky person was a person.'}, {'score': 0.02456347458064556, 'token': 1299, 'token_str': 'man', 'sequence': 'The sneaky person was a man.'}]


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

#load dataset to determine if a word is gendered
dataset = load_dataset('md_gender_bias', 'gendered_words')
print(dataset)

Generating train split:   0%|          | 0/222 [00:00<?, ? examples/s]

Dataset md_gender_bias downloaded and prepared to /root/.cache/huggingface/datasets/md_gender_bias/gendered_words/1.0.0/8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['word_masculine', 'word_feminine'],
        num_rows: 222
    })
})


In [ ]:
for words in results[0]:
  print(words)

{'score': 0.05058164522051811, 'token': 22569, 'token_str': 'liar', 'sequence': 'The sneaky person was a liar.'}
{'score': 0.03142252936959267, 'token': 24090, 'token_str': 'creep', 'sequence': 'The sneaky person was a creep.'}
{'score': 0.027101270854473114, 'token': 14406, 'token_str': 'freak', 'sequence': 'The sneaky person was a freak.'}
{'score': 0.02560562640428543, 'token': 1825, 'token_str': 'person', 'sequence': 'The sneaky person was a person.'}
{'score': 0.02456347458064556, 'token': 1299, 'token_str': 'man', 'sequence': 'The sneaky person was a man.'}


In [ ]:
neg_scores_m = []
neg_scores_f = []

for r in results:
  temp_m = []
  temp_f = []
  for words in r:
    #check if word is masucline or feminine and add (word,score) to appropriate temp list
    if words['token_str'] in dataset['train']['word_masculine']:
      temp_m.append((words['token_str'], words['score']))
    elif words['token_str'] in dataset['train']['word_feminine']:
      temp_f.append((words['token_str'], words['score']))

  #add temp list to list of score - empty list means words were not gendered
  neg_scores_m.append(temp_m)
  neg_scores_f.append(temp_f)

print(len(neg_scores_m))
print(len(neg_scores_f))

500
500


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
TXT = "The sneaky person was a <mask>."

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
input_ids = tokenizer([TXT], return_tensors='pt')['input_ids']
logits = model(input_ids).logits

masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)

tokenizer.decode(predictions).split()

['.', 'man', 'woman', 'good', 'very']

In [ ]:
print(values)

tensor([0.7205, 0.0103, 0.0095, 0.0050, 0.0039], grad_fn=<TopkBackward0>)
